In [141]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [142]:
X = np.random.randint(0,6,(5,5))
X = torch.FloatTensor(X)
X

tensor([[2., 2., 4., 5., 1.],
        [1., 4., 3., 4., 3.],
        [4., 5., 1., 0., 5.],
        [5., 0., 2., 3., 3.],
        [0., 2., 1., 1., 1.]])

In [143]:
class MFModel(nn.Module):
    def __init__(self, n_users, n_items, n_dims):
        super().__init__()
        self.users = nn.Embedding(n_users, n_dims)
        self.items = nn.Embedding(n_items, n_dims)
        self.users.weight.data.uniform_(0, 0.1)
        self.items.weight.data.uniform_(0, 0.1)
    
    def forward(self, user_ids, item_ids):
        user_embed = self.users(user_ids)
        item_embed = self.items(item_ids)
        
        predicted_ratings = (user_embed * item_embed).sum(dim=1)
        return predicted_ratings

n_users = 5
n_items = 5
n_dims = 3

model = MFModel(n_users, n_items, n_dims)

In [144]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [145]:
user_ids, item_ids = X.nonzero(as_tuple=True)
ratings = X[user_ids, item_ids]

num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    
    predictions = model(user_ids, item_ids)
    loss = loss_fn(predictions, ratings)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item():.4f}")

Epoch: 0, Loss: 10.0326
Epoch: 10, Loss: 9.6667
Epoch: 20, Loss: 8.9205
Epoch: 30, Loss: 7.7697
Epoch: 40, Loss: 6.2913
Epoch: 50, Loss: 4.6959
Epoch: 60, Loss: 3.2732
Epoch: 70, Loss: 2.2444
Epoch: 80, Loss: 1.6679
Epoch: 90, Loss: 1.4258
Epoch: 100, Loss: 1.3220
Epoch: 110, Loss: 1.2553
Epoch: 120, Loss: 1.2153
Epoch: 130, Loss: 1.1966
Epoch: 140, Loss: 1.1881
Epoch: 150, Loss: 1.1837
Epoch: 160, Loss: 1.1810
Epoch: 170, Loss: 1.1789
Epoch: 180, Loss: 1.1767
Epoch: 190, Loss: 1.1743
Epoch: 200, Loss: 1.1711
Epoch: 210, Loss: 1.1667
Epoch: 220, Loss: 1.1608
Epoch: 230, Loss: 1.1523
Epoch: 240, Loss: 1.1404
Epoch: 250, Loss: 1.1235
Epoch: 260, Loss: 1.0997
Epoch: 270, Loss: 1.0664
Epoch: 280, Loss: 1.0208
Epoch: 290, Loss: 0.9603
Epoch: 300, Loss: 0.8831
Epoch: 310, Loss: 0.7907
Epoch: 320, Loss: 0.6891
Epoch: 330, Loss: 0.5887
Epoch: 340, Loss: 0.5019
Epoch: 350, Loss: 0.4375
Epoch: 360, Loss: 0.3957
Epoch: 370, Loss: 0.3702
Epoch: 380, Loss: 0.3534
Epoch: 390, Loss: 0.3403
Epoch: 400

In [154]:
user_ids, item_ids = (X == 0).nonzero(as_tuple=True)

prediction_list = []
with torch.inference_mode():
    predictions = model(user_ids, item_ids)
    predictions = torch.clamp(predictions, min=0, max=5)

for user, item, rating in zip(user_ids, item_ids, predictions):
    print(f"Predicted rating for {user}, {item} is {rating}")

reconstructed_matrix = np.zeros_like(X.numpy())


with torch.inference_mode():
    for i in range(n_users):
        for j in range(n_items):
            user_id = torch.tensor([i])
            item_id = torch.tensor([j])
            if X[i, j] != 0:
                reconstructed_matrix[i, j] = X[i, j].item()
            else:
                predicted_rating = model(user_id, item_id)
                reconstructed_matrix[i, j] = np.round(torch.clamp(predicted_rating, min=0, max=5).detach().numpy().item())

print("Original Matrix: \n", X.numpy())
print("Reconstructed Matrix: \n", reconstructed_matrix)

Predicted rating for 2, 3 is 1.632796049118042
Predicted rating for 3, 1 is 3.9782395362854004
Predicted rating for 4, 0 is 3.732208728790283
Original Matrix: 
 [[2. 2. 4. 5. 1.]
 [1. 4. 3. 4. 3.]
 [4. 5. 1. 0. 5.]
 [5. 0. 2. 3. 3.]
 [0. 2. 1. 1. 1.]]
Reconstructed Matrix: 
 [[2. 2. 4. 5. 1.]
 [1. 4. 3. 4. 3.]
 [4. 5. 1. 2. 5.]
 [5. 4. 2. 3. 3.]
 [4. 2. 1. 1. 1.]]
